In [21]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torchvision
import torchvision.transforms as transforms
import matplotlib.pyplot as plt
import numpy as np

# Device configuration
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

# Hyper-Parameters
num_epochs = 100
batch_size = 4
learning_rate = 0.001

# Transform Data
transform = transforms.Compose([transforms.ToTensor(),transforms.Normalize((0.5,0.5,0.5),(0.5,0.5,0.5))])

train_dataset = torchvision.datasets.CIFAR10(root='./data',train = True, download = True, transform = transform)
test_dataset = torchvision.datasets.CIFAR10(root='./data',train = False, download = True, transform = transform)

train_loader = torch.utils.data.DataLoader(train_dataset,batch_size = batch_size, shuffle = True)
test_loader = torch.utils.data.DataLoader(test_dataset,batch_size = batch_size, shuffle = False)

classes = ('plane', 'car', 'bird', 'cat', 'deer', 'dog', 'frog', 'horse', 'ship', 'truck')

# Implement convnet
class ConvNet(nn.Module):
  def __init__(self):
    super(ConvNet,self).__init__()
    self.conv1 = nn.Conv2d(3,6,5)
    self.pool = nn.MaxPool2d(2,2)
    self.conv2 = nn.Conv2d(6,16,5)
    self.fc1 = nn.Linear(16*5*5,120)
    self.fc2 = nn.Linear(120,84)
    self.fc3 = nn.Linear(84,10)
    
  def forward(self,x):
    x = self.pool(F.relu(self.conv1(x)))
    x = self.pool(F.relu(self.conv2(x)))
    x = x.view(-1,16*5*5)
    x = F.relu(self.fc1(x))
    x = F.relu(self.fc2(x))
    x = self.fc3(x)
    return x

model = ConvNet().to(device)

criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(model.parameters(),lr = learning_rate)

n_total_steps = len(train_loader)

for epoch in range(num_epochs):
  for i,(images,labels) in enumerate(train_loader):
    images = images.to(device)
    labels = labels.to(device)

    # Forward Pass
    outputs = model(images)
    loss = criterion(outputs,labels)

    # Backward Pass and Weight Update
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()

    if (i+1)%2000 == 0:
      print(f'Epoch {epoch+1}/{num_epochs}, step [{i+1}/{n_total_steps}], Loss = {loss.item():.4f}')
 
print('Training Finished Successfully')



Files already downloaded and verified
Files already downloaded and verified
Epoch 1/100, step [2000/12500], Loss = 2.3238
Epoch 1/100, step [4000/12500], Loss = 2.3015
Epoch 1/100, step [6000/12500], Loss = 2.2622
Epoch 1/100, step [8000/12500], Loss = 2.2341
Epoch 1/100, step [10000/12500], Loss = 2.2479
Epoch 1/100, step [12000/12500], Loss = 2.1735
Epoch 2/100, step [2000/12500], Loss = 2.2624
Epoch 2/100, step [4000/12500], Loss = 1.8957
Epoch 2/100, step [6000/12500], Loss = 1.1462
Epoch 2/100, step [8000/12500], Loss = 1.6814
Epoch 2/100, step [10000/12500], Loss = 1.7950
Epoch 2/100, step [12000/12500], Loss = 1.5826
Epoch 3/100, step [2000/12500], Loss = 1.1113
Epoch 3/100, step [4000/12500], Loss = 2.3662
Epoch 3/100, step [6000/12500], Loss = 1.6769
Epoch 3/100, step [8000/12500], Loss = 1.2175
Epoch 3/100, step [10000/12500], Loss = 2.6856
Epoch 3/100, step [12000/12500], Loss = 1.5753
Epoch 4/100, step [2000/12500], Loss = 2.7278
Epoch 4/100, step [4000/12500], Loss = 1.137

In [22]:
# Testing
with torch.no_grad():
  n_correct = 0
  n_samples = 0
  n_class_correct = [0 for i in range(10)]
  n_class_samples =[0 for i in range(10)]

  for images,labels in test_loader:
    images = images.to(device)
    labels = labels.to(device)
    
    outputs = model(images)
    _,predicted = torch.max(outputs,1)
    n_samples += labels.size(0)
    n_correct += (predicted == labels).sum().item()

    for i in range(batch_size):
      label = labels[i]
      pred = predicted[i]
      if(label == pred):
        n_class_correct[label] += 1
      n_class_samples[label] += 1 
  
  acc = 100.0 * n_correct/n_samples
  print(f'Accuracy of the model = {acc}%')

  for i in range(10):
    acc = 100.0 * n_class_correct[i]/n_class_samples[i]
    print(f'Accuracy of {classes[i]} = {acc}%')



Accuracy of the model = 59.11%
Accuracy of plane = 67.0%
Accuracy of car = 65.7%
Accuracy of bird = 50.3%
Accuracy of cat = 46.4%
Accuracy of deer = 46.6%
Accuracy of dog = 51.1%
Accuracy of frog = 65.5%
Accuracy of horse = 64.5%
Accuracy of ship = 64.0%
Accuracy of truck = 70.0%
